In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine


### Step1:  Load and Prepare Dataset

In [18]:
import pandas as pd
import numpy as np

titanic_df = pd.read_csv('titanic.csv')

# Encode categorical variables 
titanic_df['Sex'] = titanic_df['Sex'].map({'male': 0, 'female': 1})

X = titanic_df[['Pclass', 'Sex', 'Age', 'Fare']].values
y = titanic_df['Survived'].values

classes = np.unique(y)
overall_mean = np.mean(X, axis=0).reshape(X.shape[1], 1)


### Step2: Step 2: Compute Scatter Matrices (S_W and S_B) Simultaneously

In [19]:
# Initialize scatter matrices
S_W = np.zeros((X.shape[1], X.shape[1]))
S_B = np.zeros((X.shape[1], X.shape[1]))

# Compute S_W and S_B simultaneously
for cl in classes:
    class_indices = np.where(y == cl)[0]
    class_X = X[class_indices]
    class_mean = np.mean(class_X, axis=0).reshape(X.shape[1], 1)
    
    # Within-class scatter matrix
    diff = class_X - class_mean.T
    S_W += diff.T.dot(diff)
    
    # Between-class scatter matrix
    n = class_X.shape[0]
    diff = (class_mean - overall_mean)
    S_B += n * diff.dot(diff.T)

print("Within-Class Scatter Matrix S_W:\n", S_W)
print("Between-Class Scatter Matrix S_B:\n", S_B)


Within-Class Scatter Matrix S_W:
 [[ 5.50847138e+02  1.84999041e+01 -3.73424893e+03 -1.70984184e+04]
 [ 1.84999041e+01  1.43309414e+02 -3.28596477e+02  8.98983991e+02]
 [-3.73424893e+03 -3.28596477e+02  1.55204905e+05  6.87919025e+04]
 [-1.70984184e+04  8.98983991e+02  6.87919025e+04  2.05228991e+06]]
Between-Class Scatter Matrix S_B:
 [[ 7.12763185e+01 -6.54134844e+01  2.56819919e+02 -3.22045615e+03]
 [-6.54134844e+01  6.00328978e+01 -2.35695195e+02  2.95555751e+03]
 [ 2.56819919e+02 -2.35695195e+02  9.25363042e+02 -1.16038161e+04]
 [-3.22045615e+03  2.95555751e+03 -1.16038161e+04  1.45508888e+05]]


### Step 3:  Compute Eigenvalues and Eigenvectors

In [28]:
# Compute eigenvalues and eigenvectors
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

# Sort eigenvalues and corresponding eigenvectors in descending order
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:, i]) for i in range(len(eig_vals))]
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

# Print sorted eigenvalues
print("Sorted Eigenvalues:")
for i, (val, _) in enumerate(eig_pairs):
    print(f"Eigenvalue {i+1}: {val:.2e}")

# Choose top k eigenvectors
k = 2
W_lda = np.hstack([eig_pairs[i][1].reshape(X.shape[1], 1) for i in range(k)])
print("Projection Matrix W (LDA):\n", W_lda)


Sorted Eigenvalues:
Eigenvalue 1: 6.26e-01
Eigenvalue 2: 6.20e-18
Eigenvalue 3: 6.20e-18
Eigenvalue 4: 0.00e+00
Projection Matrix W (LDA):
 [[-3.53769399e-01+0.j          7.12987928e-01+0.j        ]
 [ 9.35280362e-01+0.j          6.58586159e-01-0.23873708j]
 [-9.89158177e-03+0.j          5.01320279e-03-0.02962973j]
 [ 1.13244258e-04+0.j          2.80277982e-03+0.00248633j]]


### PCA for comparison


In [22]:
# Standardize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Print standardized data
print("Standardized Data:")
for i in range(len(X_scaled)):
    print(X_scaled[i], y[i])


Standardized Data:
[ 0.82737724 -0.73769513 -0.56243386 -0.50244517] 0.0
[-1.56610693  1.35557354  0.64625712  0.78684529] 1.0
[ 0.82737724  1.35557354 -0.26026111 -0.48885426] 1.0
[-1.56610693  1.35557354  0.41962756  0.42073024] 1.0
[ 0.82737724 -0.73769513  0.41962756 -0.48633742] 0.0
[ 0.82737724 -0.73769513  0.04191163 -0.47811643] 0.0
[-1.56610693 -0.73769513  1.8549481   0.39581356] 0.0
[ 0.82737724 -0.73769513 -2.07329759 -0.22408312] 0.0
[ 0.82737724  1.35557354 -0.18471793 -0.42425614] 1.0
[-0.36936484  1.35557354 -1.16677935 -0.0429555 ] 1.0
[ 0.82737724  1.35557354 -1.92221121 -0.31217238] 1.0
[-1.56610693  1.35557354  2.15712085 -0.11384571] 1.0
[ 0.82737724 -0.73769513 -0.71352023 -0.48633742] 0.0
[ 0.82737724 -0.73769513  0.72180031 -0.01870931] 0.0
[ 0.82737724  1.35557354 -1.16677935 -0.49027979] 0.0
[-0.36936484  1.35557354  1.93049129 -0.32626666] 1.0
[ 0.82737724 -0.73769513 -2.07329759 -0.06199889] 0.0
[-0.36936484 -0.73769513  0.04191163 -0.38667072] 1.0
[ 0.82737

### Compute covariance Matrix

In [23]:
# Compute the covariance matrix
cov_matrix = np.cov(X_scaled.T)

print("Covariance Matrix:")
print(cov_matrix)


Covariance Matrix:
[[ 1.0011236  -0.13204869 -0.35323533 -0.55011704]
 [-0.13204869  1.0011236  -0.10026143  0.1825377 ]
 [-0.35323533 -0.10026143  1.0011236   0.09773627]
 [-0.55011704  0.1825377   0.09773627  1.0011236 ]]


### Compute Eigenvalues and eigenvectors

In [29]:
# Compute eigenvalues and eigenvectors
eig_vals_pca, eig_vecs_pca = np.linalg.eig(cov_matrix)

# Sort eigenvalues and corresponding eigenvectors in descending order
eig_pairs_pca = [(np.abs(eig_vals_pca[i]), eig_vecs_pca[:, i]) for i in range(len(eig_vals_pca))]
eig_pairs_pca = sorted(eig_pairs_pca, key=lambda k: k[0], reverse=True)

# Print sorted eigenvalues
print("Sorted Eigenvalues (PCA):")
for i, (val, _) in enumerate(eig_pairs_pca):
    print(f"Eigenvalue {i+1}: {val:.2e}")

# Choose top k eigenvectors for PCA
k_pca = 2
W_pca = np.hstack([eig_pairs_pca[i][1].reshape(X.shape[1], 1) for i in range(k_pca)])
print("Projection Matrix W (PCA):\n", W_pca)


Sorted Eigenvalues (PCA):
Eigenvalue 1: 1.74e+00
Eigenvalue 2: 1.13e+00
Eigenvalue 3: 7.52e-01
Eigenvalue 4: 3.88e-01
Projection Matrix W (PCA):
 [[ 0.66961221  0.08033199]
 [-0.21940909  0.73384931]
 [-0.37178781 -0.63911132]
 [-0.60436162  0.21575141]]


### Project Data onto New Feature Subspace for PCA

In [30]:
# Project data onto new feature subspace for PCA
X_pca = X_scaled.dot(W_pca)

# Print the projected data for PCA
print("Projected Data (PCA):")
for i in range(len(X_pca)):
    print(X_pca[i], y[i])


Projected Data (PCA):
[ 1.22864355 -0.22383761] 0.0
[-2.06191909  0.62571097] 1.0
[0.64880341 1.1221164 ] 1.0
[-1.75639509  0.69156264] 1.0
[ 0.85379018 -0.84800892] 0.0
[ 0.98925191 -0.6048327 ] 0.0
[-1.81568892 -1.76728655] 0.0
[1.62213293 0.80182951] 0.0
[0.58167675 1.08777303] 1.0
[-0.08500137  1.70154909] 1.0
[1.15991645 2.2224069 ] 1.0
[-2.07929673 -0.5342245 ] 1.0
[ 1.27508072 -0.12380133] 0.0
[ 0.45882955 -0.94023952] 0.0
[0.98669738 1.70117491] 0.0
[-1.06530644 -0.33907643] 1.0
[1.52417544 0.83679941] 0.0
[ 0.13263252 -0.68123983] 1.0
[0.38577426 0.92448055] 0.0
[0.39050411 0.66041073] 1.0
[-0.16598988 -0.86616923] 0.0
[ 0.02028838 -0.87436185] 1.0
[0.95648183 1.65365462] 1.0
[-0.88634274 -0.58307353] 1.0
[0.99433454 2.04829025] 0.0
[0.02626445 0.64467347] 1.0
[ 1.00425948 -0.61019026] 0.0
[-3.40193223  0.83973084] 0.0
[0.76170488 1.31503947] 1.0
[ 0.99609676 -0.60727625] 0.0
[-1.12871295 -1.19623312] 0.0
[-2.90724146  1.07325048] 1.0
[0.76327706 1.31447821] 1.0
[-0.84804319 -